In [ ]:
# Всё необходимое для выполнения данной лабораторной работы!
import requests
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Задание 1. Парсинг

В этом задании Вам предстоит работать с API сайта <a target="_blank" href="hh.ru">hh.ru</a>. Документация доступна по <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">ссылке</a>.

**Задача:**
1. выбрать 5 интересующих вас профессий (не смежных, т.е. Аналатик и Data Engineer не считается корректным)
2. затем получить доступные по данным профессиям вакансии в Москве и выделить их физические координаты
3. нанести полученные координаты на карту местоположения компаний

*Пример запроса для профессии:*

In [ ]:
URL = 'https://api.hh.ru/vacancies'

params = {
    'text': "Физик-ядерщик",
    'area': 1,
    'page': 0,
    'per_page': 10
}

data = requests.get(URL, params).json()
data

{'items': [],
 'found': 0,
 'pages': 1,
 'page': 0,
 'per_page': 10,
 'clusters': None,
 'arguments': None,
 'fixes': None,
 'suggests': None,
 'alternate_url': 'https://hh.ru/search/vacancy?area=1&enable_snippets=true&items_on_page=10&order_by=publication_time&text=%D0%A4%D0%B8%D0%B7%D0%B8%D0%BA-%D1%8F%D0%B4%D0%B5%D1%80%D1%89%D0%B8%D0%BA'}

In [ ]:
# формирую список описаний вакансий, описание которых включает ключевое, переданное в поле text
# GET https://api.hh.ru/vacancies?text=архитектор&area=1&page=0&per-page=7


URL = 'https://api.hh.ru/vacancies/'

params = {'text': "архитектор",
          'area': 1,
          'page': 0,
          'per-page': 7}

# получаю список описаний вакансий от URL с данными параметрами params
data = requests.get(URL, params=params).json()

Если Ваш запрос прошел успешно, то вы получите следующие ключи (значения данных ключей вы найдете по ссылке на документацию):

In [ ]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

☑

Выберете для вас 5 наиболее интересных специальностей и получите по ним имеющиеся вакансии:

**выбранные профессии:**

1.   аналитик данных
2.   художник
3.   психолог
4.   экономист
5.   архитектор



Получаю вакансии, соответствующие списку специальностей

```
specialties = ['аналитик данных', 'художник', 'психолог','экономист','архитектор']
```
структура на выходе: список словарей



In [ ]:
URL = 'https://api.hh.ru/vacancies/'

specialties, responses = ['аналитик данных', 'художник', 'психолог','экономист','архитектор'], []

for i in specialties:
  params = {'text': i,
            'area': 1,
            'page': 0,
            'per-page': 7
            }

  response = requests.get(url=URL, params=params).json()
  cnt_vacancies = len(response)
  responses.append(response)

Посчитайте сколько всего найдено вакансий:

In [ ]:
# пример запроса к api hh

URL = 'https://api.hh.ru/vacancies/'
params = {
    'text': "аналитик данных",  # Кавычки внутри строки не нужны
    'area': 1,                  # 1 — Москва
    'page': 0,                  # API нумерует страницы с 0
    'per_page': 100
}

response = requests.get(URL, params)
data = response.json() # множество всех найденных вакансий с их параметрами
total_pages = data['pages'] # количество страниц с вакансиями

print(f'Количество страниц с результатами: {total_pages}')

Количество страниц с результатами: 20


In [ ]:
# подсчет через произведение кол-ва страниц и количества вакансий на одну страницу
# ---------------------------------------

cnt_vacancies = total_pages * params['per_page']
print(f'Количество вакансий: {cnt_vacancies}')

Количество вакансий: 2000


In [ ]:
# подсчет кол-ва вакансий через размер списка id вакансий

UNIQUE_ID = [] # список id уникальных вакансий


for page in range(total_pages):
  params['page'] = page

  response = requests.get(URL, params)
  data = response.json() # переменная data хранит описание всех найденных вакансий в формате json

  if 'items' in data:
    for vacancy in data['items']:
      UNIQUE_ID.append(vacancy['id'])

print(f'Количество уникальных вакансий: {len(UNIQUE_ID)}')

Количество уникальных вакансий: 2000


**Создайте структуру данных, которая будет хранить в себе имя компании и ее координаты (широта и долгота):**

In [ ]:
# структура

about_company = {
    'company_name':None,
    'lat': None,
    'lng': None
    }

**пример парсинга данных о компании**

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
def get_vacancy_info(id):
    response = requests.get(f'https://api.hh.ru/vacancies/{id}/')
    data = response.json()
    company_name = data.get('department', {})
    address = data.get('address')
    if address and address.get('lat') and address.get('lng'):
        return {'name': company_name, 'lat': address['lat'], 'lng': address['lng']}
    return None

vacancies_data = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(get_vacancy_info, id) for id in UNIQUE_ID]
    for future in as_completed(futures):
        result = future.result()
        if result:
            vacancies_data.append(result)

In [ ]:
vacancies_data

[{'name': None, 'lat': 55.656194, 'lng': 37.526169},
 {'name': None, 'lat': 55.796764, 'lng': 37.599013},
 {'name': None, 'lat': 55.69351, 'lng': 37.660548},
 {'name': None, 'lat': 55.770335, 'lng': 37.590847},
 {'name': None, 'lat': 55.76288, 'lng': 37.615264},
 {'name': None, 'lat': 55.766131, 'lng': 37.604205},
 {'name': None, 'lat': 55.801131, 'lng': 37.58411},
 {'name': None, 'lat': 55.781489, 'lng': 37.572118},
 {'name': None, 'lat': 55.780846, 'lng': 37.570051},
 {'name': None, 'lat': 55.747282, 'lng': 37.539078},
 {'name': None, 'lat': 55.788353, 'lng': 37.567931},
 {'name': None, 'lat': 55.753904, 'lng': 37.51529},
 {'name': None, 'lat': 55.797007, 'lng': 37.938118},
 {'name': None, 'lat': 55.852733, 'lng': 37.390218},
 {'name': None, 'lat': 55.733416, 'lng': 37.636931},
 {'name': None, 'lat': 55.762272, 'lng': 37.569899},
 {'name': None, 'lat': 55.729696, 'lng': 37.636311},
 {'name': None, 'lat': 55.745726, 'lng': 37.66088},
 {'name': None, 'lat': 55.748179, 'lng': 37.540245}

In [ ]:
def get_vacancy_info(id):
  response = requests.get(url=f'https://api.hh.ru/vacancies/{id}')
  data = response.json()
  company_name = data.get('department', {})
  return company_name, id

company_names = []
for id in UNIQUE_ID[:10]:
  print(get_vacancy_info(id))

({'id': 'mailru-15478-vkdigit', 'name': 'VK, VK Tech'}, '121950103')
(None, '122530028')
(None, '122723102')
(None, '122724683')
(None, '122703571')
(None, '122514685')
(None, '121559769')
(None, '122560150')
({}, '122639109')
({}, '122701655')


Создайте экземпляр этой структуры данных для каждой компании, которая выставила вакансию:

In [ ]:
# структура данных для хранения вакансий компаний

сompany_vacancies = [
    {
        'name': 'Company_Name',
        'lat': None,
        'lng': None,
        'vacancies': [
            {
               'id': 'input_id',
               'title': 'input_title',
               'experience': '4-7 age',
               'skills': [],
               'salary': {'from': None, 'to': None, 'currency': 'EUR'}
            },
            {}
        ]
    }
]

**Найдите количество вакансий, у которых заданы координаты (значения данных может быть пустым):**

In [ ]:
cnt_vacancies_with_address = len(vacancies_data)
print(f'Количество вакансий с указанными координатами: {cnt_vacancies_with_address}')

Количество вакансий с указанными координатами: 127


**🤓 Отберите вакансии, которые расположены в пределах МКАД (можно грубо):**

In [ ]:
# import requests
# from tqdm import tqdm
# import time

# def fetch_mkad_vacancies(max_pages=10, key_value='МКАД'):
#   URL = "https://api.hh.ru/vacancies/"
#   use_params = {
#       'text': key_value,
#       'area': 1,
#       'per_page':100,
#       'page': 0
#   }

#   mkad_vacancies = []

#   # инициализирую сессию
#   session = requests.Session()

#   with tqdm(desc="поиск вакансий с ключевым словом МКАД") as pbar:
#     while True:
#       try:
#         response = session.get(url=URL, params=use_params, timeout=10)
#         response.raise_for_status()
#         data = response.json()

#         for v in data['items']:
#           mkad_vacancies.append({
#               'id': v['id'],
#               'name': v['name'],
#               'company': v.get('employer', {}).get('name'),
#               'url': v.get('alternate_url')
#           })

#         params['page'] += 1
#         pbar.update(1)

#         if params['page'] >= data['pages'] or params['page'] >= max_pages:
#           break

#         time.sleep(0.1)

#       except requests.exceptions.RequestException as e:
#         print(f'Сетевая ошибка: {e}')
#         break
#       except ValueError as e:
#         print(f'Ошибка при обработке JSON: {e}')
#         break
#       except Exception as e:
#         print(f'Иная ошибка: {e}')
#         break

#   return mkad_vacancies


In [ ]:
# print(fetch_mkad_vacancies())

поиск вакансий с ключевым словом МКАД: 1it [00:01,  1.38s/it]

[{'id': '122531890', 'name': 'Шеф-повар', 'company': 'Иванова Юлия', 'url': 'https://hh.ru/vacancy/122531890'}, {'id': '122532393', 'name': 'Директор по персоналу (HRD)', 'company': 'Березина-Лебедева Ирина', 'url': 'https://hh.ru/vacancy/122532393'}, {'id': '122553907', 'name': 'Домработница/Помощница (Домработник/Помощник) по хозяйству', 'company': 'Прямой работодатель.', 'url': 'https://hh.ru/vacancy/122553907'}, {'id': '122459652', 'name': 'HR Generalist с ростом до HRD', 'company': 'Парипа Екатерина Николаевна', 'url': 'https://hh.ru/vacancy/122459652'}, {'id': '122680405', 'name': 'Руководитель отдела безопасности', 'company': '5POST', 'url': 'https://hh.ru/vacancy/122680405'}, {'id': '122711774', 'name': 'Директор по маркетингу / Руководитель отдела маркетинга', 'company': 'Опт-Юнион', 'url': 'https://hh.ru/vacancy/122711774'}, {'id': '122519722', 'name': 'Курьер пеший', 'company': 'ФинЭкспертиза', 'url': 'https://hh.ru/vacancy/122519722'}, {'id': '122438571', 'name': 'Няня', 'c

In [ ]:
# парсинг вакансий с ключевым словом и загрузка результата в csv-файл

import requests, time
from tqdm import tqdm
import csv

# парсинг вакансий
def fetch_vacancies(max_pages=4):
  URL = "https://api.hh.ru/vacancies/"
  use_params = {
      'text': "МКАД",
      'area': 1,
      'per_page': 100,
      'page': 0
  }

  relevant_vacancies = []
  session = requests.Session()

  for page in tqdm(range(max_pages), desc='Загрузка вакансий'):
    use_params['page'] = page
    response = requests.get(url=URL, params=use_params, timeout=10)
    data = response.json()

    for v in data['items']:
      relevant_vacancies.append({
          'id': v['id'],
          'name': v['name'],
          'company': v.get('employer', {}).get('name'),
          'url': v.get('alternate_url')
      })

    if page >= data['pages'] - 1:
      break
    time.sleep(0.2)

  return relevant_vacancies


def get_coordinates(vacancy_id):
  url = f'https://api.hh.ru/vacancies/{vacancy_id}'
  response = requests.get(url, timeout=10).json()

  address = response.get('address')
  if address:
    return address.get('lat'), address.get('lng')
  return None, None

def is_inside_mkad(lat, lng):
  from math import radians, cos, sin, sqrt, atan2
  center_lat, center_lng = 55.751244, 37.618423
  radius_mkad_km = 18

  if lat is None or lng is None:
    return False

  R = 6371
  dlat = radians(lat - center_lat)
  dlng = radians(lng - center_lng)
  a = sin(dlat/2)**2 + cos(radians(center_lat)) * cos(radians(lat)) * sin(dlng/2)**2

  c = 2 * atan2(sqrt(a), sqrt(1-a))

  distance = R * c
  return distance <= radius_mkad_km

def save_to_csv(data, filename='vacancies_inside_mkad.csv'):
  with open(filename, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

vacancies = fetch_vacancies(max_pages=5)

filtered_vacancies = []
for v in tqdm(vacancies, desc='Проверка координат компаний'):
  lat, lng = get_coordinates(v['id'])
  if is_inside_mkad(lat, lng):
    v['lat'] = lat
    v['lng'] = lng

    filtered_vacancies.append(v)
  time.sleep(0.1)

# запись найденных вакансий в CSV-файл
save_to_csv(filtered_vacancies)
print(f'Вакансий {len(filtered_vacancies)} сохранено в csv-файл')

Проверка координат компаний: 100%|██████████| 500/500 [05:15<00:00,  1.58it/s]

Вакансий 212 сохранено в csv-файл


Постройте график в координатах "широта-долгота" для отображения вакансий внутри МКАД (не забывайте прописать прозрачность, чтобы увидеть скопления):

**Сделайте выводы о проделанной работе:**

# Задание 2. Анализ данных датасета diamonds

Вам предстоит проанализировать датасет, содержащий информацию об бриллианта, их стоимости и остальных характеристиках.



Поясним значения хранящиеся в колонках:
*   `cut` - огранка бриллианта (относится к одной из примерно 10 наиболее распространенных)
*   `color` - цвет бриллианта, прозрачные бриллианты имеют градацию `D-Z`, "более высокие" буквы - более желтоватые, но часто имеют лучшие значения, поскольку цвет трудно определить один раз в кольце
*   `clarity` - чистота бриллианта, наличие включений (внутренних дефектов), чем меньше и мельче, тем лучше
*   `carat_weight` - масса бриллианта (слабо связана с размером бриллианта, т.к. огранка и качество огранки, как правило, играют большую роль)
*   `cut_quality` - качество ограники по системе оценки GIA Cut
*   `lab` - лаборатория оценивания
*   `meas_length` - длина бриллианта
*   `meas_width` - ширина бриллианта
*   `meas_depth` - глубина бриллианта
*   `total_sales_price` - цена бриллианта в долларах.
*    **и прочие колонки**

Скачаем данные:

In [ ]:
!gdown 10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8
# если работайте в jupyter то скачайте датасет с https://drive.google.com/uc?id=10OJ30qlkE-7zK4JuVTDMY3U4nRB4tu_8

Создадим DataFrame:

In [ ]:
from google.colab import drive
drive.mount("content/")

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [ ]:
url_dataset = "/content/content/MyDrive/Поступашки/Аналитика/Семинар 1 Парсинг/diamonds.csv"
diamonds = pd.read_csv(url_dataset)

Выведете первые 5 строк датасета:

In [ ]:
display(diamonds.head())

,Unnamed: 0,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,...,meas_depth,girdle_min,girdle_max,fluor_color,fluor_intensity,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,total_sales_price
0,0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.79,M,M,unknown,NaN,unknown,unknown,unknown,unknown,200
1,1,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,...,1.78,STK,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
2,2,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.77,TN,M,unknown,NaN,unknown,unknown,unknown,unknown,200
3,3,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,...,1.78,M,STK,unknown,NaN,unknown,unknown,unknown,unknown,200
4,4,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Excellent,unknown,...,1.82,STK,STK,unknown,NaN,unknown,unknown,unknown,unknown,200


**Найдите количество бриллиантов всего в данном датасете:**

In [ ]:
print(f'кол-во бриллиантов в датасете:', len(diamonds))

кол-во бриллиантов в датасете: 219703


**Выведите максимальную и минимальную цену бриллианта в формате `номер - цена`:**

In [ ]:
max_price, max_index = diamonds["total_sales_price"].max(), diamonds['total_sales_price'].idxmax()
min_price, min_index  = diamonds['total_sales_price'].min(), diamonds['total_sales_price'].idxmin()

print(f"min value index - {min_index}, price - {min_price}")
print(f"max value index - {max_index}, price - {max_price}")

min value index - 0, price - 200
max value index - 219702, price - 1449881


**Посчитайте количество пропусков (пропуском считается значение `unknown` или `None`):**

In [ ]:
int((diamonds.isna().sum() | diamonds.isin(["unknown"]).sum()).sum())

1890464

**Найдите в каких столбцах присутствуют пропуски:**

In [ ]:
diamonds.columns[(diamonds.isna().sum() > 0) | (diamonds.eq("unknown").sum())].to_list()

['color',
 'cut_quality',
 'eye_clean',
 'culet_size',
 'culet_condition',
 'girdle_min',
 'girdle_max',
 'fluor_color',
 'fluor_intensity',
 'fancy_color_dominant_color',
 'fancy_color_secondary_color',
 'fancy_color_overtone',
 'fancy_color_intensity']

В нашем датасете присутствуют колонки, которые начинаются с `fancy_color_` или `fluor_`. Большинство значений в этих колонках - `unknown` или `None`, поэтому мы считаем их малоинформативными.

Ваша задача:

**найти количество таких колонок, которые начинаются с `fancy_color_` или `fluor_`:**

In [ ]:
fancy_fluor_cols = [col for col in  diamonds.columns if col.startswith("fancy_color_") | (col.startswith("fluor_"))]
len(fancy_fluor_cols)

6

**найти количество информативных (не `None` или `unknown`) значений в этих колонках:**

In [ ]:
inf_values = {}
total_count = 0

for col in fancy_fluor_cols:
  inf_values[col] = diamonds[col].apply(lambda x: str(x).lower()!="unknown" and pd.notna(x)).sum()
  print(f'{col} - {inf_values[col]}')
  total_count += inf_values[col]

print(f'\ntotal count - {total_count}')

fluor_color - 15726
fluor_intensity - 76084
fancy_color_dominant_color - 9164
fancy_color_secondary_color - 1062
fancy_color_overtone - 388
fancy_color_intensity - 9162

total count - 111586


удалить данные колонки из датасета, сохранив изменения (в дальнейшем работаем измененным датасетом):

In [ ]:
diamonds.drop(columns=fancy_fluor_df, inplace=True)

Проделайте тоже самое для колонок, начинающихся с `culet_`:

In [ ]:
# поиск атрибутов по маске
culet_cols = [col for col in diamonds.columns if col.startswith("culet_")]
print(f"найдено {len(culet_cols)} столбца: {culet_cols}")

найдено 2 столбца: ['culet_size', 'culet_condition']


In [ ]:
# вывести кол-во информативных значений для каждого атрибута
count_culet_inform_values = {}

for col in culet_cols:
  count_culet_inform_values[col] = diamonds[col].apply(lambda x: x.lower()!= "unknown" and pd.notna(x)).sum()
  print(f'{col} - {count_culet_inform_values[col]}')

culet_size - 133963
culet_condition - 15319


In [ ]:
# удалить атрибуты из рабочего датафрейма
diamonds.drop(columns=culet_cols, inplace=True)

Посчитайте сколько бы стоил (в тысячах долларов) сейф, в котором хранились бы все бриллианты из датасета:

In [ ]:
float(diamonds['total_sales_price'].sum(axis=0))

1517721991.0

Немного необоснованных манипуляций.

Выберите из датасета строки с четными индексами и выведете значения (не меняя нумерацию индексов):

In [ ]:
diamonds[::2].head()

,Unnamed: 0,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,depth_percent,table_percent,meas_length,meas_width,meas_depth,girdle_min,girdle_max,total_sales_price
0,0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,62.7,59.0,2.85,2.87,1.79,M,M,200
2,2,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,61.1,59.0,2.88,2.90,1.77,TN,M,200
4,4,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Excellent,unknown,64.9,58.5,2.79,2.83,1.82,STK,STK,200
6,6,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,64.0,57.0,2.85,2.88,1.84,STK,STK,200
8,8,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,63.5,59.5,2.89,2.92,1.85,STK,TK,200


А теперь с индексами, кратными 6:

In [ ]:
diamonds[::6].head()

,Unnamed: 0,cut,color,clarity,carat_weight,cut_quality,lab,symmetry,polish,eye_clean,depth_percent,table_percent,meas_length,meas_width,meas_depth,girdle_min,girdle_max,total_sales_price
0,0,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,62.7,59.0,2.85,2.87,1.79,M,M,200
6,6,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,64.0,57.0,2.85,2.88,1.84,STK,STK,200
12,12,Round,E,VVS2,0.09,Very Good,IGI,Very Good,Very Good,unknown,61.8,58.5,2.85,2.89,1.77,STK,TK,200
18,18,Round,E,VVS2,0.09,Excellent,IGI,Very Good,Very Good,unknown,61.0,60.0,2.91,2.92,1.78,M,M,200
24,24,Round,L,I1,0.34,Excellent,IGI,Excellent,Excellent,unknown,0.0,0.0,0.00,0.00,0.00,unknown,unknown,204


Выберете из датасета только категорильные признаки и выведете датасет:

In [ ]:
categorial_cols = [col for col in diamonds.columns if np.dtype(diamonds[col]) == object]
diamonds[categorial_cols]

,cut,color,clarity,cut_quality,lab,symmetry,polish,eye_clean,girdle_min,girdle_max
0,Round,E,VVS2,Excellent,IGI,Very Good,Very Good,unknown,M,M
1,Round,E,VVS2,Very Good,IGI,Very Good,Very Good,unknown,STK,STK
2,Round,E,VVS2,Excellent,IGI,Very Good,Very Good,unknown,TN,M
3,Round,E,VVS2,Excellent,IGI,Very Good,Very Good,unknown,M,STK
4,Round,E,VVS2,Very Good,IGI,Very Good,Excellent,unknown,STK,STK
...,...,...,...,...,...,...,...,...,...,...
219698,Round,E,VS1,Excellent,GIA,Excellent,Excellent,unknown,M,STK
219699,Radiant,unknown,VS2,unknown,GIA,Very Good,Very Good,unknown,TK,XTK
219700,Round,E,VS1,Excellent,GIA,Excellent,Excellent,unknown,TN,M
219701,Princess,unknown,SI2,unknown,GIA,Good,Good,unknown,XTN,VTK


### Построение графиков

**Требования к графику:**

1.   График должен быть подписан
2.   Оси должны быть подписаны
3.   Должна быть легенда графика
4.   График должен быть информативным (желательно без сильных выбросов)

**Все надписи на русском языке!**

>**Примечание**
>
>График необходимо строить для очищенных данных - в них не должно быть пропусков по осям, по которым строите график. При этом **не перезатирайте** имеющийся датафрейм!

Постройте график `цвет бриллианта - стоимость`:

In [ ]:
# собираю сводную таблицу
color_price = diamonds.pivot_table(index='color',
                                       values='total_sales_price',
                                       aggfunc=['min','mean', 'median','max']).rename(columns={'total_sales_price':'tsp'})

color_price.columns = ['_'.join(col).strip() for col in color_price]

color_price.sort_index(inplace=True)
color_price.reset_index(inplace=True)
color_price

,color,min_tsp,mean_tsp,median_tsp,max_tsp
0,D,208,6262.167039,1584.0,1121792
1,E,200,5321.490409,1242.0,1315496
2,F,200,5603.393937,1510.0,1097128
3,G,212,7156.929927,2331.0,1003311
4,H,220,7598.123308,2692.0,1132347
5,I,212,6853.358165,2294.0,566093
6,J,212,6489.142680,1998.0,557292
7,K,252,6280.997277,1630.0,424418
8,L,204,5949.089917,1890.0,390096
9,M,240,5987.224811,2280.0,285416


Отфильтровываю из датафрейма записи с "аномальными"(*) значениями mean_tsp согласно логике:
```
если средняя цена по столбцу mean_tsp меньше:
  mean_tsp < mean(mean_tsp) - 1.5 * std(mean_tsp)
  mean_tsp > mean(mean_tsp) + 1.5 * std(mean_tsp)
то, удалить mean_tsp
```

In [ ]:
mean_mean_price, std_mean_price = color_price['mean_tsp'].mean(), color_price['mean_tsp'].std()
under_price, upper_price = mean_mean_price - 1.5 * std_mean_price, mean_mean_price + 1.5 * std_mean_price

filtered_color_price = color_price[(color_price['mean_tsp'] < upper_price) & (color_price['mean_tsp'] > under_price)]
filtered_color_price

,color,min_tsp,mean_tsp,median_tsp,max_tsp
0,D,208,6262.167039,1584.0,1121792
1,E,200,5321.490409,1242.0,1315496
2,F,200,5603.393937,1510.0,1097128
3,G,212,7156.929927,2331.0,1003311
4,H,220,7598.123308,2692.0,1132347
5,I,212,6853.358165,2294.0,566093
6,J,212,6489.142680,1998.0,557292
7,K,252,6280.997277,1630.0,424418
8,L,204,5949.089917,1890.0,390096
9,M,240,5987.224811,2280.0,285416


In [ ]:
import plotly.express as px

scatter_fig = px.scatter(
    data_frame=filtered_color_price,
    x='color',
    y='mean_tsp',
    color='color',
    hover_data=['color','min_tsp','mean_tsp', 'median_tsp', 'max_tsp']

)
scatter_fig.update_layout(
    xaxis_title='color name',
    yaxis_title = 'mean price, $',
    title='Price diamonds by colors'
)

scatter_fig.update_traces(
    marker=dict(size=9)
)

scatter_fig.show()

Постройте график `лабараторная оценка - кол-во бриллиантов`:

In [ ]:
# агрегация собрана из исходного датафрейма diamonds
lab_perfomance = diamonds[['lab']].groupby(by='lab').agg({'lab':'count'}).rename(columns={'lab':'num_analysis'}).sort_values(by='num_analysis', ascending=False)

# агрегация собрана без учета записей для бриллиантов с color="unknown"
df_clear = diamonds.query('color!="unknown"')
lab_perfomance_clear = df_clear[['lab']].groupby(by='lab').agg({'lab':'count'}).rename(columns={'lab':'num_analysis'}).sort_values(by='num_analysis', ascending=False)


In [ ]:
import plotly.express as px

pie_fig = px.pie(
    data_frame=lab_perfomance,
    names=lab_perfomance.index,
    values=lab_perfomance['num_analysis'],
    color=lab_perfomance.index,
    color_discrete_map={'IGI':'#FFCE72',
                        'GIA':'#FFA2BF',
                        'HRD':'#6D5DD2'},
    hole=.5
)

pie_fig.update_layout(
    title={
        'text':"Number of analyses by mark's system",
        'x':.5,
        'y':.95,
        'xanchor':'center',
        'yanchor':'top',
        'font':{
            'family':'Monospace',
            'size':18,
            'color':'black'
        }
    }
)

pie_fig.show()

Постройте график `вес бриллианта - стоимость`:

In [ ]:
carat_price_df = diamonds[['carat_weight', 'total_sales_price']]
carat_price_df.rename(columns={'total_sales_price':'tsp'}, inplace=True)

carat_price_df.loc[:, 'weight_range'] = np.where(
    carat_price_df['carat_weight'] <= 5, 'w1',
    np.where((carat_price_df['carat_weight'] > 5) & (carat_price_df['carat_weight'] <= 10), 'w2',
             np.where((carat_price_df['carat_weight'] > 10) & (carat_price_df['carat_weight'] <= 15), 'w3','w4')
             )
    )

pivot_carat_price_df = carat_price_df.pivot_table(index=['weight_range', 'carat_weight'], values='tsp', aggfunc=['min', 'mean', 'median','max'])
pivot_carat_price_df.columns = ['_'.join(col).strip() for col in pivot_carat_price_df.columns]
pivot_carat_price_df.reset_index(inplace=True)

In [ ]:
scatter_fig = px.scatter(
    data_frame=pivot_carat_price_df,
    x='weight_range',
    y='median_tsp',
    color='weight_range',
    hover_data=['median_tsp'],

    )

scatter_fig.update_layout(
    xaxis_title='weight range',
    yaxis_title='median sales price, $',
    title={
        'text':'Median salling price ($) by weight range',
        'x':.5,
        'y':.95,
        'xanchor':'center',
        'yanchor':'top',
        'font':{
            'family':'Monospace',
            'size':18,
            'color':'black'
        },
    }
)

scatter_fig.show()

Постройте график `система оценки GIA Cut - кол-во бриллиантов`:

In [ ]:
quality_lab_df = diamonds.pivot_table(index=['lab','cut_quality'], aggfunc={'cut_quality':'count'}).rename(columns={'cut_quality':'count'})
quality_lab_df.reset_index(inplace=True)

quality_lab_df

,lab,cut_quality,count
0,GIA,Excellent,112816
1,GIA,Fair,3
2,GIA,Good,6
3,GIA,Very Good,29903
4,GIA,unknown,57706
5,HRD,Excellent,1890
6,HRD,Good,2
7,HRD,Very Good,568
8,HRD,unknown,944
9,IGI,Excellent,10155


In [ ]:

# xaxis_title='quality',
# yaxis_title='number diamonds'

import plotly.express as px

bar_fg = px.bar(
    data_frame=quality_lab_df,
    y=['lab'],
    x='count',
    color='cut_quality'
    )

bar_fg.update_layout(
    xaxis_title='laboratory',
    yaxis_title='count diamonds',
    title={
        'text':'Number of diamonds by type of proportions (cut quality)',
        'x':.5,
        'y':.95,
        'xanchor':'center',
        'yanchor':'top',
        'font':{
            'family':'Monospace',
            'size':18,
            'color':'black'
        },
    }

)

bar_fg.show()